In [1]:
import random

import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm
import pickle

import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
parent_dir = os.path.dirname(parent_dir)
sys.path.insert(0, parent_dir)

from explain.baseline_quantile import NeutralFairBaseline
from explain.other_baselines import maximum_distance_bs
from explain.other_baselines import p_data_bs
from explain.sampling_shapley_mod import shapley_sampling
from explain.sparse_mlp import get_sparse_mlp
from explain.mlp import validate_mlp

In [2]:
seed = 3
np.random.seed(seed)
random.seed(seed)
samples = 300

# Download and Preprocess

In [3]:
# Load CSV from URL using NumPy
url = "https://raw.githubusercontent.com/meauxt/credit-card-default/master/credit_cards_dataset.csv"
names = ['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'Default']
df = pd.read_csv(url, names=names)
df = df.drop(['ID'], axis = 1)
names = names[1:np.size(names,0)]
print(df.shape)

# Convert to numeric
for column in df:
    df[column] = pd.to_numeric(df[column],errors='coerce')
df = df.dropna()

# rescale sex (sex : male = 0 , female = 1)
df['SEX'][df['SEX']==1]=0
df['SEX'][df['SEX']==2]=1

# rescale marital status (married = 0, single = 1) after dropping 'others'
df['MARRIAGE'][df['MARRIAGE']==1]=0
df['MARRIAGE'][df['MARRIAGE']==2]=1
df.drop(df[df['MARRIAGE']==3].index , inplace=True)

# remove unknown from education
df.drop(df[df['EDUCATION']==5].index, inplace=True)
df.drop(df[df['EDUCATION']==6].index, inplace=True)
print("Dataset:")
print(df.shape)

# Convert in log PAY_AMT
pay_atm_vars = ['PAY_AMT1','PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
for j in pay_atm_vars:
    # set 0 to min
    df[j][df[j]==0] = np.min(df[j][df[j]!=0])
    # take log
    df[j] = np.log(df[j])
    
# balanced subsampling
index_sample = np.random.choice(df[df['Default']==1].index.values, size=samples // 2, replace=False)
index_sample = np.hstack((np.random.choice(df[df['Default']==0].index.values, size=samples // 2, replace=False), index_sample))
np.random.shuffle(index_sample)
df = df.loc[index_sample]

arr_df = df.values
all_keys = df.keys()

# Scale all data
scaler = MinMaxScaler()
scaler.fit(arr_df)
arr_df = scaler.transform(arr_df)
outcome = np.asarray(arr_df[:, -1])
data = np.asarray(arr_df[:, :-1])
x_train, x_test, y_train, y_test = train_test_split(data, outcome, test_size=0.2, random_state=seed)
n_vars = x_test.shape[1]
n_out_of_sample = x_test.shape[0]
print("Training set: ", x_train.shape[0], " (", np.sum(y_train), ")") 
print("Testing set: ", n_out_of_sample, " (", np.sum(y_test), ")") 
print('n features: ', n_vars)

(30001, 24)
Dataset:
(29351, 24)
Training set:  240  ( 115.0 )
Testing set:  60  ( 35.0 )
n features:  23


# Validate Best Model and Compute Performance on testset

In [4]:
# Params for validation
runs = 300
hidden_layers = range(0, 6)
nodes_hidden_layers = range(1, 11, 1)

# Validate
model, conf, performance = validate_mlp(x_train, y_train, runs, hidden_layers, nodes_hidden_layers)

# Get Predictions
y_hat_train = model.predict(x_train)
y_hat_test = model.predict(x_test)
roc_auc = roc_auc_score(np.array(y_test), y_hat_test)
ave_pre = average_precision_score(np.array(y_test), y_hat_test)

print('Best model: ')
print(conf)
print('Test set results: ')
print("ROC AUC: ", roc_auc)
print("AVG PRE: ", ave_pre)


Best model: 
((9, 'sigmoid'), (1, 'sigmoid'))
Test set results: 
ROC AUC:  0.7805714285714286
AVG PRE:  0.864737020111747


# Compute Feature Attributions using different Baselines

In [5]:
runs_sh_sampling = 2**7

# Get weights and biases and store them in a list
ws = []
bs = []
for layer in model.layers:
    ws.append(layer.get_weights()[0])
    bs.append(layer.get_weights()[1])

ls = []
for _, activation in conf:
    ls.append(activation)

## Zero Baseline

In [6]:
%%time
zero_reference = np.zeros_like(x_test)[0, :]
a_zero = np.array([shapley_sampling(model,
                           x_test[i],
                           y_hat_test[i],
                           runs = runs_sh_sampling,
                           baseline = zero_reference) for i in tqdm(range(x_test.shape[0]))])
# training data
a_zero_train = np.array([shapley_sampling(model,
                           x_train[i],
                           y_hat_train[i],
                           runs = runs_sh_sampling,
                           baseline = zero_reference) for i in tqdm(range(x_train.shape[0]))])


Wall time: 5h 54min 41s


## Average

In [7]:
%%time
average_reference = np.mean(x_train, 0)
a_average = np.array([shapley_sampling(model,
                              x_test[i],
                              y_hat_test[i],
                              runs = runs_sh_sampling,
                              baseline = average_reference) for i in tqdm(range(x_test.shape[0]))])
# training data
a_average_train = np.array([shapley_sampling(model,
                              x_train[i],
                              y_hat_train[i],
                              runs = runs_sh_sampling,
                              baseline = average_reference) for i in tqdm(range(x_train.shape[0]))])


Wall time: 5h 41min 30s


## Neutral (0.5)

In [8]:
%%time
# Find baseline
nf_bas = NeutralFairBaseline()
reference, errors_list = nf_bas.search_baseline_mlp(ws, bs, ls, np.array(x_train))
# Get Sparse Model
model_sparse = get_sparse_mlp(ws, bs, ls, reference)
# Apply Shapley
a_neutral_05 = np.array([shapley_sampling(model_sparse,
                                 x_test[i],
                                 y_hat_test[i],
                                 runs = runs_sh_sampling,
                                 baseline = reference) for i in tqdm(range(x_test.shape[0]))])
# training data
a_neutral_05_train = np.array([shapley_sampling(model_sparse,
                                 x_train[i],
                                 y_hat_train[i],
                                 runs = runs_sh_sampling,
                                 baseline = reference) for i in tqdm(range(x_train.shape[0]))])

Prediction at the reference point is:  [[0.5004683]]



Wall time: 6h 4min 4s


## Maximum Distance

In [9]:
%%time

maxdist_references = np.zeros_like(x_test)
for i in range(x_test.shape[0]):
    maxdist_references[i] = maximum_distance_bs(x_train, x_test[i])

maxdist_references_train = np.zeros_like(x_train)
for i in range(x_train.shape[0]):
    maxdist_references_train[i] = maximum_distance_bs(x_train, x_train[i])

a_maxdist = np.zeros_like(x_test)
for i in tqdm(range(x_test.shape[0])):
    a_maxdist[i] = shapley_sampling(model,
                                    x_test[i],
                                    y_hat_test[i],
                                    runs = runs_sh_sampling,
                                    baseline = maxdist_references[i])
# training data
a_maxdist_train = np.zeros_like(x_train)
for i in tqdm(range(x_train.shape[0])):
    a_maxdist_train[i] = shapley_sampling(model,
                                    x_train[i],
                                    y_hat_train[i],
                                    runs = runs_sh_sampling,
                                    baseline = maxdist_references_train[i])


Wall time: 5h 53min 57s


## P Data

In [10]:
%%time
n_draws = 10
pdata_references = p_data_bs(x_train, seed = 1, n_draws = n_draws)

a_pdata = np.zeros_like(x_test)
def comp_a_pdata(draw):
    return np.array([shapley_sampling(model,
                                      x_test[i],
                                      y_hat_test[i],
                                      runs = runs_sh_sampling,
                                      baseline = pdata_references[draw]) for i in range(x_test.shape[0])])
for draw in tqdm(range(n_draws)):
    a_pdata += comp_a_pdata(draw)
a_pdata /= n_draws


a_pdata_train = np.zeros_like(x_train)
def comp_a_pdata(draw):
    return np.array([shapley_sampling(model,
                                      x_train[i],
                                      y_hat_train[i],
                                      runs = runs_sh_sampling,
                                      baseline = pdata_references[draw]) for i in range(x_train.shape[0])])
for draw in tqdm(range(n_draws)):
    a_pdata_train += comp_a_pdata(draw)
a_pdata_train /= n_draws


Wall time: 2d 12h 33min 45s


# Pickle outputs

In [11]:
# pickle model
with open("conf.pickle", "wb") as f:
    pickle.dump(conf, f)
model.save('model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model\assets


In [12]:
# pickle references 
with open('zero_reference.npy', 'wb') as f:
    np.save(f, zero_reference)
with open('average_reference.npy', 'wb') as f:
    np.save(f, average_reference)
with open('reference.npy', 'wb') as f:
    np.save(f, reference)
with open('maxdist_references.npy', 'wb') as f:
    np.save(f, maxdist_references)
    np.save(f, maxdist_references_train)

In [13]:
# pickle attributions 
with open('a_zero.npy', 'wb') as f:
    np.save(f, a_zero)
    np.save(f, a_zero_train)
with open('a_average.npy', 'wb') as f:
    np.save(f, a_average)
    np.save(f, a_average_train)
with open('a_neutral_05.npy', 'wb') as f:
    np.save(f, a_neutral_05)
    np.save(f, a_neutral_05_train)
with open('a_maxdist.npy', 'wb') as f:
    np.save(f, a_maxdist)
    np.save(f, a_maxdist_train)
with open('a_pdata.npy', 'wb') as f:
    np.save(f, a_pdata)
    np.save(f, a_pdata_train)